In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'new-plant-diseases-dataset' dataset.
Path to dataset files: /kaggle/input/new-plant-diseases-dataset
Path to dataset files: /kaggle/input/new-plant-diseases-dataset


In [10]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50, DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os
import numpy as np

In [11]:
# Load and Preprocess Dataset
dataset_path = path  # Path from kagglehub download
image_size = (224, 224)  # Standard size for most models
batch_size = 32

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
    )

# Training and validation generators
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
    )

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
    )

Found 140615 images belonging to 3 classes.
Found 35152 images belonging to 3 classes.
Found 35152 images belonging to 3 classes.


In [12]:
# Define Models
def create_model(base_model):
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize models
models = {
    'VGG16': create_model(VGG16(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'VGG19': create_model(VGG19(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'InceptionV3': create_model(InceptionV3(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'Xception': create_model(Xception(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'ResNet50': create_model(ResNet50(weights='imagenet', include_top=False, input_shape=image_size + (3,))),
    'DenseNet121': create_model(DenseNet121(weights='imagenet', include_top=False, input_shape=image_size + (3,)))
}

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Train Models
history = {}
epochs = 10

for model_name, model in models.items():
    print(f"Training {model_name}...")
    history[model_name] = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs
    )
    model.save(f"{model_name}_plant_disease_model.h5")
    print(f"{model_name} training completed and model saved.")

Training VGG16...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2275/4395 ━━━━━━━━━━━━━━━━━━━━ 19:00 538ms/step - accuracy: 0.4987 - loss: 0.7102

In [ ]:
# Evaluate Models
evaluation_results = {}

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    evaluation_results[model_name] = model.evaluate(val_generator)
    print(f"{model_name} evaluation completed.")

# Display Results
for model_name, result in evaluation_results.items():
    print(f"{model_name}: Loss = {result[0]}, Accuracy = {result[1]}")